## Preprocesamiento:  

Este código carga imágenes de dos carpetas diferentes ("Datanoface" y "Dataface"), las redimensiona a un tamaño determinado, las normaliza y las etiqueta según la carpeta de origen. Luego divide los datos y las etiquetas en tres conjuntos: un conjunto de prueba, un conjunto de entrenamiento y un conjunto de validación. Finalmente, devuelve los seis conjuntos de datos y etiquetas en variables separadas: X_test, y_test, X_train, y_train, X_val y y_val.  

En resumen, este código se utiliza para cargar, procesar y dividir un conjunto de imágenes en tres conjuntos diferentes para entrenar, validar y evaluar modelos de aprendizaje automático o de visión por computadora.


## Datos

Fotos con rostro: 202.999 imagenes.
Fotos sin rostro: 32.461 imagenes.



## La arquitectura de este modelo es la siguiente:  

Se carga el modelo pre-entrenado VGG16 sin la capa superior (include_top=False) y se congela su pesos para que no se actualicen durante el entrenamiento.  

Se agrega una nueva capa superior al modelo para la clasificación binaria. Primero se aplana la salida de la base del modelo usando la capa Flatten(). Luego se agrega una capa Dense de 128 unidades con función de activación relu, seguida de una capa de salida Dense con una única unidad y función de activación sigmoid.  

Se compila el modelo con la función de pérdida binary_crossentropy, el optimizador Adam y la métrica de precisión (accuracy).  

Finalmente, se entrena el modelo durante 15 épocas con un tamaño de lote (batch size) de 100 y se utiliza EarlyStopping para prevenir el sobreajuste. Después, se evalúa el modelo en el conjunto de pruebas.
    
## Pruebas  
La función llamada "predecir_con_modelo" se utiliza en el  modelo de aprendizaje  para hacer predicciones sobre la presencia de caras en imágenes. La función toma dos argumentos: el modelo y la ruta de la carpeta que contiene las imágenes que se van a analizar.

La función utiliza un bucle "for" para recorrer cada imagen en la carpeta. Para cada imagen, se carga la imagen utilizando la librería OpenCV, se realiza el preprocesamiento (cambio de tamaño y normalización) y se utiliza el modelo para hacer una predicción sobre si hay o no una cara en la imagen.

Si la predicción es mayor a 0.01, la función muestra un mensaje indicando que hay una cara en la imagen, de lo contrario muestra un mensaje indicando que no hay una cara en la imagen.



In [6]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split


def etiquetar_imagenes(folder1, folder2, img_size=100, test_size=0.2):
    data = []
    labels = []

    for category in os.listdir(folder1):
        path = os.path.join(folder1, category)
        if os.path.isdir(path):
            for img_name in os.listdir(path):
                img_path = os.path.join(path, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB color space
                    img = cv2.resize(img, (img_size, img_size))
                    img = img.astype('float32') / 255.0  # Normalize the image
                    data.append(img)
                    labels.append(0)
                else:
                    print(f"Failed to load image: {img_path}")

    for category in os.listdir(folder2):
        path = os.path.join(folder2, category)
        if os.path.isdir(path):
            for img_name in os.listdir(path):
                img_path = os.path.join(path, img_name)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB color space
                    img = cv2.resize(img, (img_size, img_size))
                    img = img.astype('float32') / 255.0  # Normalize the image
                    data.append(img)
                    labels.append(1)
                else:
                    print(f"Failed to load image: {img_path}")

    data = np.array(data).reshape(-1, img_size, img_size, 3)  # Reshape to 4D array
    labels = np.array(labels)

    print(f"Loaded {len(data)} images.")
    
    # Split the data and labels into training and validation sets
    X_train_val, X_test, y_train_val, y_test = train_test_split(data, labels, test_size=test_size, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=test_size, random_state=42)

    return X_test, y_test, X_train, y_train, X_val, y_val

X_test, y_test, X_train, y_train, X_val, y_val = etiquetar_imagenes(r".\Datanoface", r".\Dataface")




Failed to load image: .\Datanoface\noface\ILSVRC2012_val_00013491.JPEG
Failed to load image: .\Datanoface\noface\ILSVRC2012_val_00028437.JPEG
Loaded 235458 images.


In [7]:

#import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import patlib
import os
import keras
from keras import layers, models
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Dropout,Activation,MaxPooling2D,Flatten
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Load the pre-trained VGG16 model without the top layer (include_top=False)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(100, 100, 3))

# Freeze the weights of the pre-trained layers so they are not updated during training
for layer in base_model.layers:
    layer.trainable = False

# Add a new top layer to the model for binary classification
x = Flatten()(base_model.output)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=output)

# Compile the model with binary cross-entropy loss and Adam optimizer
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()

early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
# Train the model for 10 epochs with a batch size of 32
H=model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=15, batch_size=100,callbacks=[early_stop])

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)



Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0     

In [9]:
model.save('modeloVGG16.h5')

In [23]:
import os

def predecir_con_modelo(modelo, folder_path):
    # Recorrer todas las imágenes en la carpeta
    for filename in os.listdir(folder_path):
        img_path = os.path.join(folder_path, filename)
        # Cargar la imagen y hacer la preprocesamiento
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convertir a espacio de color RGB
        img = cv2.resize(img, (100, 100))  # Cambiar el tamaño a 100x100 píxeles
        img = img.astype('float32') / 255.0  # Normalizar la imagen

        # Hacer la predicción
        pred = modelo.predict(np.array([img]))
        if pred[0][0] > 0.01:
            print(filename, "Sí hay una cara en la imagen.")
        else:
            print(filename, "No hay una cara en la imagen.")

In [24]:
predecir_con_modelo(model, './fotos/')

1/1 [==============================] - 1s 802ms/step
FACE1.jpg Sí hay una cara en la imagen.
1/1 [==============================] - 1s 790ms/step
FACE2.jpg Sí hay una cara en la imagen.
1/1 [==============================] - 1s 792ms/step
FACE3.jpg Sí hay una cara en la imagen.
1/1 [==============================] - 1s 785ms/step
FACE4.jpg Sí hay una cara en la imagen.
1/1 [==============================] - 1s 783ms/step
FACE5.jpg Sí hay una cara en la imagen.
1/1 [==============================] - 1s 796ms/step
FACE6.jpg Sí hay una cara en la imagen.
1/1 [==============================] - 1s 796ms/step
FACE7.jpg Sí hay una cara en la imagen.
1/1 [==============================] - 1s 795ms/step
FACE8.jpg No hay una cara en la imagen.
1/1 [==============================] - 1s 792ms/step
FACE9.jpg Sí hay una cara en la imagen.
1/1 [==============================] - 1s 791ms/step
NOFACE.jpg No hay una cara en la imagen.
1/1 [==============================] - 1s 792ms/step
NOFACE1.jpg No h